# W24 CS 480/680 assignment 1 (coding part)


- Please save a copy of this notebook to avoid losing your changes.
- Debug your code and ensure that it can run before submission.
- Save the output of each cell. Failure to do so may result in your coding questions not being graded.
- Submit your completed version of this notebook.



## Implement Your Own Perceptron

In this question, you are asked to implement the **Perceptron** algorithm and test with a classification task on the adult dataset. Please refer to the [dataset webpage](https://archive.ics.uci.edu/dataset/53/iris) for details on the dataset, and consult the lecture slides and suggested readings for details on the Perceptron algorithm.

- `scikit-learn` or other off-the-shelf Perceptron libraries are not allowed to use.

- Please note that `class` column is the label of the dataset. The labellings have three possible values. So you need to implement a multi-class extension of Perceptron.

- Recording the accuracy on the training dataset when you train the Perceptron model on the training data, as an iteration. Repeat this step for 10 times.

- It is required that you split the dataset into training/testing datasets (80% and 20%, respectively), and we will deduct marks if you don't.

- You will get full marks for
    - Correctly split data into training set and testing set.
    - Correctly implement the perceptron algorithm.
    - Correctly handle multiple class.
    - A plot on training accuracy vs number of iterations, that relects the progress of the training.
    - Final reported accuracy (on the test dataset).

In [1]:
import urllib.request
import pandas as pd
# TODO: add any other package you need
import numpy as np

# Download the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
filename = 'iris.csv'
urllib.request.urlretrieve(url, filename) 

# Load the dataset into a Pandas dataframe
column_names = [
    'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'
]
data = pd.read_csv(filename, names=column_names)


In [2]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Code for dataset splitting (training-testing):

In [3]:
len(data)

150

In [4]:
data[:5]

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
train_size = int(len(data)*0.8)
test_size = int(-len(data)*0.2)

In [6]:
# TODO: your implementation

train = data[:train_size]
test = data[test_size:]

In [7]:
test

,sepal_length,sepal_width,petal_length,petal_width,class
120,6.9,3.2,5.7,2.3,Iris-virginica
121,5.6,2.8,4.9,2.0,Iris-virginica
122,7.7,2.8,6.7,2.0,Iris-virginica
123,6.3,2.7,4.9,1.8,Iris-virginica
124,6.7,3.3,5.7,2.1,Iris-virginica
125,7.2,3.2,6.0,1.8,Iris-virginica
126,6.2,2.8,4.8,1.8,Iris-virginica
127,6.1,3.0,4.9,1.8,Iris-virginica
128,6.4,2.8,5.6,2.1,Iris-virginica
129,7.2,3.0,5.8,1.6,Iris-virginica


In [9]:
test.shape

(30, 5)

Code for implemeting the Perceptron algorithm:
- You can either implement it as an class/object or implement the training function and the prediction function.

In [148]:
class Perceptron():
    def __init__(self, w, b):
        #padding trick
        self.w = w
        self.b = b

    def fit(self, X, y, iterations):
        #init w vector
        self.w = np.zeros(X.shape[1]) + self.w
        #pad
        self.w = np.append(self.w, self.b)

        for i in range(iterations):
            for idx, j in enumerate(X):
                #pad x
                j = np.append(j, 1)
                print(f'Weights n bias {self.w}')
                print(f'score{(np.inner(j, self.w))}')
                if (y[idx]*(np.inner(j, self.w)) <= 0):
                    self.w = self.w + y[idx]*j

    def predict(self, X):
        X = np.array([np.append(i, 1) for i in X])
        print(X.shape)
        return np.dot(self.w, X.T)
        



In [149]:
for enumerate?

Init signature: enumerate(iterable, start=0)
Docstring:     
Return an enumerate object.

  iterable
    an object supporting iteration

The enumerate object yields pairs containing a count (from start, which
defaults to zero) and a value yielded by the iterable argument.

enumerate is useful for obtaining an indexed list:
    (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
Type:           type
Subclasses:     

In [150]:

X = np.array([[1,1,0,1,1 ],
             [0,0,1,1,0],
             [0,1,1,0,0],
             [1,0,0,1,0],
             [ 1,0,1,0,1],
             [1,0,1,1,0]])
y = np.array([1, -1, 1, -1, 1, -1])




In [151]:
per = Perceptron(0, 0)

In [152]:
per.fit(X, y, 1)

Weights n bias [0. 0. 0. 0. 0. 0.]
score0.0
Weights n bias [1. 1. 0. 1. 1. 1.]
score2.0
Weights n bias [ 1.  1. -1.  0.  1.  0.]
score0.0
Weights n bias [1. 2. 0. 0. 1. 1.]
score2.0
Weights n bias [ 0.  2.  0. -1.  1.  0.]
score1.0
Weights n bias [ 0.  2.  0. -1.  1.  0.]
score-1.0


In [127]:
x_test = np.array([[0,0,0]])

In [128]:
per.predict(x_test)

(1, 4)


array([0.])

In [26]:
vector = np.array([1, 2, 3])

# Define a 3x4 matrix
matrix = np.array([[4, 5, 6, 7],
                   [8, 9, 10, 11],
                   [12, 13, 14, 15]])

# Compute the dot product
result = np.dot(vector, matrix)

# Display the result
print("Resulting 1x4 vector:", result)

Resulting 1x4 vector: [56 62 68 74]


In [27]:
np.matmul(w, X)

array([10,  6,  6])

Code for implementing a wrapper function:
- Handling multi-class extension
- Feeding in data and call Perceptron training function
- Outputing multiple Perceptrons that are trained

In [18]:
# TODO: your implementation
def multiclass(func):

SyntaxError: incomplete input (2722723362.py, line 2)

Code for testing and ploting:



In [6]:
# TODO: plot the accuracy against the number of iterations
# TODO: report the final accuracy